In [50]:
import numpy as np
import sys
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import random
from tqdm import tqdm_notebook as tqdm
from progressbar import ProgressBar

IN_IPYNB = True


import networkx as nx


def init_array(size):
    A = np.random.randint(low=1, high=100, size=(size, size))
    np.fill_diagonal(A, 0)
    A = np.triu(A) + np.tril(A.T)
    return A


def TSP_ph1(Graph, S, destination):
    if (len(S) <= 0):
        #s=list(S)
        return list([(1, destination)]), Graph[0, destination-1]
    else:
        min_dist = sys.maxsize
        min_way = list()
        S=S.difference({destination})
        for _, node in enumerate(S):
            w, d = TSP_ph1(Graph, S.difference({node}), node)
            d += Graph[node - 1, destination - 1]
            if d < min_dist:
                min_dist = d
                min_way = w
                min_way.append((node, destination))
        return min_way, min_dist

def TSP(Graph):
    nodecount=len(Graph[0])
    S = set(range(2, nodecount+1))
    min_dist = sys.maxsize
    min_way = list()
    for _,node in enumerate(S):
        w, d = TSP_ph1(Graph, S, node)
        d += Graph[node - 1, 0]
        if d < min_dist:
            min_dist = d
            min_way = w
            min_way.append((node, 1))
    return min_way, min_dist


In [54]:

class Ant:
    def __init__(self, colony):
        self.colony = colony
        self.tabu = list()
        self.all_nodes = set()
        self.position = -1
        self.L = 0

    def probability(self, j):
        if j in self.tabu:
            return 0
        denominator = sys.float_info.epsilon
        for k in self.all_nodes.difference(set(self.tabu)):
            denominator += self.colony.tau[self.position, k] ** self.colony.alpha * \
                           (1 / self.colony.Graph[self.position, k]) ** self.colony.beta
        p: float
        p = self.colony.tau[self.position, j] ** self.colony.alpha * \
            (1 / self.colony.Graph[self.position, j]) ** self.colony.beta
        return p / denominator

    def move(self):
        prob = [self.probability(x) for x in self.all_nodes]
        s = sum(prob)
        if s != 1.:
            prob = [x / s for x in prob]
        nextpos = np.random.choice(list(self.all_nodes), p=prob)
        self.tabu.append(nextpos)
        self.L += self.colony.Graph[self.position, nextpos]
        self.position = nextpos
        return nextpos

    def close_circuit(self):
        if self.all_nodes.difference(set(self.tabu)) == set():
            self.L += self.colony.Graph[self.position, self.tabu[0]]

    def get_delta_tau(self):
        delta_tau_k = np.zeros(self.colony.Graph.shape).astype(float)
        for i, node in enumerate(self.tabu):
            next_node = self.tabu[i + 1] if i < len(self.tabu) - 1 else self.tabu[0]
            delta_tau_k[node, next_node] = AntColony.Q / self.L
            delta_tau_k[next_node, node] = AntColony.Q / self.L
        return delta_tau_k


class AntColony:
    q = 1.5
    Q = 100
    nr_ants = 5
    N_MAX = 1000
    TAU_ZERO = 1e-5

    def __init__(self, alpha, beta, rho):
        self.rho = rho
        self.alpha = alpha
        self.beta = beta
        self.ants = list()
        self.tau = None
        self.nr_nodes = None
        for i in range(self.nr_ants):
            self.ants.append(Ant(self))

    def loadGraph(self, Graph):
        self.Graph = Graph
        self.tau = np.zeros(Graph.shape) + self.TAU_ZERO
        self.delta_tau = np.zeros(Graph.shape)
        self.nr_nodes = Graph.shape[0]
        for ant in self.ants:
            while True:
                pos = random.randint(0, self.nr_nodes - 1)  # Nodes are 1..N , Graph is 0..N-1
                if not self.checkPosition(pos):
                    break
            ant.tabu = list()
            ant.position = pos
            ant.tabu.append(pos)
            ant.all_nodes = set(range(0, self.nr_nodes))

    def checkPosition(self, node):
        for ant in self.ants:
            if ant.position == node:
                return True
        return False

    def getBestCircuit(self):
        min = sys.maxsize
        min_way = list()
        for ant in self.ants:
            if ant.L < min:
                min_way = ant.tabu
                min = ant.L

        return min_way, min

    def checkStagnation(self):
        prev = None
        for ant in self.ants:
            if prev == None:
                prev = ant
                continue
            if prev.L != ant.L :
                return False
            t=np.array(ant.tabu)
            e=False
            for n in range(0, self.nr_nodes):
                if np.array_equal(np.array(prev.tabu), t):
                    e=True
                    break
                t=np.roll(t,1)
            if not e:
                return False
            prev = ant
        return True

    def move_all_ants(self):
        for n in range(0, self.nr_nodes - 1):  # -1 because we initalize the ants with one node
            for ant in self.ants:
                ant.move()
        for ant in self.ants:
            ant.close_circuit()

    def update_delta_tau(self):
        for ant in self.ants:
            self.delta_tau += ant.get_delta_tau()

    def reset_ants(self):
        for ant in self.ants:
            if len(ant.tabu) > 0 :
                ant.position = ant.tabu[0]
                ant.tabu = [ant.position]
            else:
                ant.position = -1
                ant.tabu = list()
            ant.L = 0

    def run(self):
        n_iteration = 0
        best_way = list()
        best_cost = sys.maxsize

        if IN_IPYNB:
            pbar = ProgressBar(maxval=self.N_MAX)
            pbar.start()
        else:
            pbar = tqdm(total=self.N_MAX)

        while True:
            self.move_all_ants()
            way, cost = self.getBestCircuit()
            if cost < best_cost:
                best_cost = cost
                best_way = way

            self.update_delta_tau()

            self.tau = self.tau * self.rho + self.delta_tau+sys.float_info.epsilon
            n_iteration += 1
            pbar.update(1)

            self.delta_tau = np.zeros(self.Graph.shape).astype(float)

            if n_iteration >= self.N_MAX or self.checkStagnation():
                break
            self.reset_ants()

        if IN_IPYNB:
            pbar.finish()
        else:
            pbar.close()
            
        res_way = []
        p = None
        for n in best_way:
            if p == None:
                p = n
                continue
            res_way.append((p + 1, n + 1))
            p = n
        res_way.append((best_way[-1] + 1, best_way[0] + 1))
        return res_way, best_cost



In [3]:
import plotly.graph_objects as go

import networkx as nx

def plot_TSP(Graph,way,way_len):
    G = nx.from_numpy_array(Graph)
    G.edges(data=True)
    pos = nx.circular_layout(G)
    print(G)

    edge_label_trace = go.Scatter(
        x=[],
        y=[],
        text=[],
        textposition='middle center',
        mode='markers+text',
        hoverinfo='none',
        marker=go.scatter.Marker(
            opacity=0
        ),
        textfont=dict(size=9, color='blue')
    )


    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

        #invisible node at middle of edges to place edge labels
        edge_label_trace['x']+=((x0+x1)/2,)
        edge_label_trace['y']+=((y0+y1)/2,)
        edge_label_trace['text']+=(Graph[edge[0]][edge[1]],)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    way_x = []
    way_y = []
    arrows = []
    for w in way:
        x0, y0 = pos[w[0]-1]
        x1, y1 = pos[w[1]-1]
        way_x.append(x0)
        way_x.append(x1)
        way_x.append(None)
        way_y.append(y0)
        way_y.append(y1)
        way_y.append(None)
        arrows.append( go.layout.Annotation(
                    x=x1,
                    y=y1,
                    xref="x",
                    yref="y",
                    text=None,
                    showarrow=True,
                    arrowhead=7,
                    ax=x0-x1,
                    ay=y0-y1
                ))    

    way_trace = go.Scatter(
        x=way_x, y=way_y,
        line=dict(width=2, color='#F00'),
        hoverinfo='none',
        mode='lines')


    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='none',
        textposition='middle center',
        marker=dict(
            showscale=False,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='Earth',
            reversescale=True,
            color='#FFF',
            size=25,
            line_width=2))
    node_trace.text = [x+1 for x in list(G.nodes())]



    fig = go.Figure(data=[edge_trace, node_trace,edge_label_trace,way_trace],
                 layout=go.Layout(
                    title='Traveling Salesman exact Solution',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    annotations=[ dict(
                        text=("Traveling Salesman result : %s, Length=%d" % (way,way_len)),
                        showarrow=False,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002 ) ].extend(arrows),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    fig.show()

In [4]:
G1 = np.array([[ 0, 43, 18, 34, 67, 41, 32,  1, 84],
       [43,  0, 93, 51,  1, 86, 44, 92, 27],
       [18, 93,  0, 49, 33, 87, 36, 22,  2],
       [34, 51, 49,  0, 18,  9, 24, 19, 28],
       [67,  1, 33, 18,  0, 89, 85, 64, 57],
       [41, 86, 87,  9, 89,  0, 27,  9,  5],
       [32, 44, 36, 24, 85, 27,  0, 42, 10],
       [ 1, 92, 22, 19, 64,  9, 42,  0, 44],
       [84, 27,  2, 28, 57,  5, 10, 44,  0]])

In [5]:
G1=init_array(9)
G2=init_array(10)
G3=init_array(11)


In [6]:

optimum={
    'G1':dict(zip(['way','cost'],TSP(G1))),
    'G2':dict(zip(['way','cost'],TSP(G2))),
    'G3':dict(zip(['way','cost'],TSP(G3))),
    
}

In [7]:
optimum

{'G1': {'way': [(1, 6),
   (6, 3),
   (3, 7),
   (7, 4),
   (4, 9),
   (9, 5),
   (5, 8),
   (8, 2),
   (2, 1)],
  'cost': 179},
 'G2': {'way': [(1, 6),
   (6, 3),
   (3, 9),
   (9, 7),
   (7, 10),
   (10, 8),
   (8, 5),
   (5, 2),
   (2, 4),
   (4, 1)],
  'cost': 208},
 'G3': {'way': [(1, 10),
   (10, 4),
   (4, 7),
   (7, 5),
   (5, 11),
   (11, 8),
   (8, 3),
   (3, 6),
   (6, 9),
   (9, 2),
   (2, 1)],
  'cost': 153}}

# Optimal Solutions

In [8]:
plot_TSP(G1,optimum['G1']['way'],optimum['G1']['cost'])

In [9]:
plot_TSP(G2,optimum['G2']['way'],optimum['G2']['cost'])

In [10]:
plot_TSP(G3,optimum['G3']['way'],optimum['G3']['cost'])

# Ant Colony Simulations

In [11]:
# alpha, beta, rho
params=[
    [1,    1,     0.50],    # Experiment 1
    [0.5,  2,     0.30],    # Experiment 2
    [0,    5,     0.70],    # Experiment 3
    [1,    0,     0.99],    # Experiment 4
    [2,    3,     0.30],    # Experiment 5
    [3,    2,     0.70],    # Experiment 6
    [2,    2,     0.50],    # Experiment 7
    [5,    1,     0.30],    # Experiment 8
    
]


In [49]:
"Experiment #%d \n  \U000003b1=%.2f, \U000003b2=%.2f, \U000003c1=%.2f" % (experiment+1,0.5,2,0.33333)

'Experiment #8 \n  α=0.50, β=2.00, ρ=0.33, '

In [55]:
NR_TRIES=10
results={
    'header':['Experiment','G1','G2','G3'],
    'values':[['Exact Solution',optimum['G1']['cost'],optimum['G2']['cost'],optimum['G3']['cost']]],
    'paths':[['Exact Solution',optimum['G1']['way'],optimum['G2']['way'],optimum['G3']['way']]]
}
for experiment,p in enumerate(params):
    cost1=sys.maxsize
    cost2=sys.maxsize
    cost3=sys.maxsize
    print("Experiment #%d" % (experiment+1))
    
    print("    G1")
    for i in range(0,NR_TRIES):
        ac = AntColony(p[0],p[1],p[2]) 
        ac.loadGraph(G1)
        pt, c = ac.run()
        if cost1>c:
            path1,cost1=pt,c
        

    print("    G2")
    for i in range(0,NR_TRIES):
        ac = AntColony(p[0],p[1],p[2]) 
        ac.loadGraph(G2)
        pt, c = ac.run()
        if cost2>c:
            path2,cost2=pt,c

    print("    G3")
    for i in range(0,NR_TRIES):
        ac = AntColony(p[0],p[1],p[2]) 
        ac.loadGraph(G3)
        pt, c = ac.run()
        if cost3>c:
            path3,cost3=pt,c
    
    results['values'].append(
        ["Experiment #%d \n  \U000003b1=%.2f, \U000003b2=%.2f, \U000003c1=%.2f" % (experiment+1,p[0],p[1],p[2]),cost1,cost2,cost3]
    )
    results['paths'].append(
        ["Experiment #%d \n  \U000003b1=%.2f, \U000003b2=%.2f, \U000003c1=%.2f" % (experiment+1,p[0],p[1],p[2]),path1,path2,path3]
    )    
    

Experiment #1
    G1
    G2
    G3
Experiment #2
    G1
    G2
    G3
Experiment #3
    G1
    G2
    G3
Experiment #4
    G1
    G2
    G3
Experiment #5
    G1
    G2
    G3
Experiment #6
    G1
    G2
    G3
Experiment #7
    G1
    G2
    G3
Experiment #8
    G1
    G2
    G3


100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################

In [72]:
results['values'].insert(0,['Exact Solution',optimum['G1']['cost'],optimum['G2']['cost'],optimum['G3']['cost']])
results['paths'].insert(0,['Exact Solution',optimum['G1']['way'],optimum['G2']['way'],optimum['G3']['way']])


# Graph G1

In [73]:

fig = go.Figure(data=[go.Table(
    header=dict(values=['Experiment','Circuit cost Graph 1'],
                line_color='darkslategray',
                fill_color='lightskyblue',
                align='left'),
    cells=dict(values=np.array([[row[0],row[1]] for row in results['values']]).T,
               line_color='darkslategray',
               fill_color='lightcyan',
               align='left'))
])

fig.update_layout(width=800, height=500)
fig.show()

# Graph G2

In [74]:

fig = go.Figure(data=[go.Table(
    header=dict(values=['Experiment','Circuit cost Graph 2'],
                line_color='darkslategray',
                fill_color='lightskyblue',
                align='left'),
    cells=dict(values=np.array([[row[0],row[2]] for row in results['values']]).T,
               line_color='darkslategray',
               fill_color='lightcyan',
               align='left'))
])

fig.update_layout(width=800, height=500)
fig.show()

# Graph G3

In [75]:

fig = go.Figure(data=[go.Table(
    header=dict(values=['Experiment','Circuit cost Graph 3'],
                line_color='darkslategray',
                fill_color='lightskyblue',
                align='left'),
    cells=dict(values=np.array([[row[0],row[3]] for row in results['values']]).T,
               line_color='darkslategray',
               fill_color='lightcyan',
               align='left'))
])

fig.update_layout(width=800, height=500)
fig.show()

# Graph Comparison 

In [77]:

fig = go.Figure(data=[go.Table(
    header=dict(values=results['header'],
                line_color='darkslategray',
                fill_color='lightskyblue',
                align='left'),
    cells=dict(values=np.array(results['values']).T,
               line_color='darkslategray',
               fill_color='lightcyan',
               align='left'))
])

fig.update_layout(width=1000, height=1000)
fig.show()